# Fitting a physical model using Gamera

## Context

[Gamera](http://libgamera.github.io/GAMERA/docs/documentation.html) is a source modelling code for gamma ray astronomy that allows for the calculation of gamma-ray spectra from underlying populations of leptonic and hadronic cosmic rays. Fitting the resulting spectra to data would directly allow to constrain the parameters of these underlying distributions.

## Approach

There are two ways in which Gamera can be used in combination with gammapy to fit the spectra of gamma-ray sources:

* Subclassing `~gammapy.modeling.models.SpectralModel` and creating a `GameraSpectralModel`. This is probbaly the most elegant way to combine Gamera and gammapy, but can also be slow because it requires the repeated execution of Gamera for every model evaluation
* Creating a `~gammapy.modeling.models.TemplateNDSpectralModel` from Gamera spectra produced on a grid of model parameters. While less elegant than the first approach, it can be significantly faster because no Gamera evaluations are required during the fit.

For this tutorial we will use the second one, but an example of the first will also be provided at the end of the notebook, for reference.


## Setup

As always, we start the notebook with some setup and imports. The environment variable `GAMERA_LIB_PATH` should point to the direcotry containing the installed Gamera library.

In [1]:
import sys
import os
sys.path.append('path/to/your/locally/installed/GAMERA/lib')
import gappa as gp

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
import itertools
from regions import CircleSkyRegion


from gammapy.maps import MapAxis,RegionNDMap,RegionGeom
from gammapy.modeling.models import (
    SpectralModel,
    SkyModel,
    SPECTRAL_MODEL_REGISTRY,
    TemplateNDSpectralModel,
    PointSpatialModel,
    Models
)
from gammapy.modeling import Parameter

## Approach 1: TemplateNDSpectralModel

The first option is to fill a grid of spectra dependent on the parameters to be fit. To do this, one just needs to create a function that takes in the parameters that we wish to fit and outputs the flux in the right units.

For this hands-on session, we will fit a leptonic model to the Crab data using two free parameters: the electron normalization and spectral index. Adding more free parameters is just a matter of exposing them as inputs to the function.

Following the previous examples, you can use the CMB as target photon field, although note a custom one can be used too.

We will use a fixed magnetic field of B=100$\mu$G

To do this, we need to define a function with the inputs/outputs:

```
def model(energy, index, effic):
    energy_shape=energy.shape

    # Define fixed quantities
    # remember that GAMERA doesn't know about units!
    distance =(2000*u.pc).to_value('pc')
    total_luminosity = 5e38*u.erg/u.s
    B = (100*u.uG).to_value('G')
    t_max = (1e3*u.yr).to_value('yr')

    # Derive luminosity from efficiency
    luminosity = (effic * total_luminosity).to_value('erg s-1')
...
    # You keep going here!!
...
    return sed.to("1 / (cm2 eV s)").reshape(energy_shape)
```

In [2]:
def gamera_model(energy, index, effic):
    energy_shape=energy.shape
    photon_energies = energy.flatten().to_value('erg')

    # Define fixed quantities
    # remember that GAMERA doesn't know about units!
    distance =(2000*u.pc).to_value('pc')
    total_luminosity = 5e38*u.erg/u.s
    B = (100*u.uG).to_value('G')
    t_max = (1000*u.yr).to_value('yr')

    # Derive luminosity from efficiency
    luminosity = (effic * total_luminosity).to_value('erg s-1')

    # Define electron energies
    e_min = -5
    e_max= 4
    energy_electrons_edges = np.logspace(e_min, e_max, 500+1)*u.TeV
    energy_electrons = MapAxis.from_edges(energy_electrons_edges, interp='log', unit='TeV')
    energy_in_erg = energy_electrons.center.to_value('erg')

    # Prepare electron power law
    fu = gp.Utils()
    power_law = (energy_in_erg **-index)
    E_cut=1*u.PeV
    exp_cutoff = np.exp(-(energy_in_erg/E_cut.to('erg')).value)
    power_law *= exp_cutoff
    integral = fu.Integrate(list(zip(energy_in_erg, power_law * energy_in_erg)))
    power_law *= luminosity / integral
    # Bundle together in the way GAMERA wants
    power_law_spectrum = np.array(list(zip(energy_in_erg, power_law)))

    # Get the relevant GAMERA modules
    fp = gp.Particles()
    fp.ToggleQuietMode()
    fp.SetSolverMethod(1)

    # Add interstellar radiation field from Popescu et al
    rad_field = np.loadtxt("models/radiation_field.txt")
    RADIATION_FIELD = list(zip(rad_field[:,0], rad_field[:,1]))
    fp.AddArbitraryTargetPhotons(RADIATION_FIELD)
    # Add the CMB as target
    fp.AddThermalTargetPhotons(2.5,0.25*gp.eV_to_erg)
    # Add FIR target
    fp.AddThermalTargetPhotons(70,0.5*gp.eV_to_erg)

    # Set particles, distance and fields
    fp.SetCustomInjectionSpectrum(power_law_spectrum)  # the injection rate is constant and given by the normalization of injected PL
    fp.SetBField(B)

    #Run the evolution of the electron spectrum and set the output as parent population for radiation calculation
    fp.SetAge(t_max)
    fp.CalculateElectronSpectrum()
    sp = np.array(fp.GetParticleSpectrum())
    
    fr = gp.Radiation()
    fr.ToggleQuietMode()
    fr.SetBField(B)
    # Add radiation
    fr.AddArbitraryTargetPhotons(fp.GetTargetPhotons()) # output from 'Particles' is in the right format to be used in 'Radiation'
    fr.SetDistance(distance)
    fr.SetElectrons(sp)


    # compute the gamma-ray spectrum on the points given by the data
    fr.CalculateDifferentialPhotonSpectrum(photon_energies)


    fr.AddSSCTargetPhotons(0.1)
    fr.CalculateDifferentialPhotonSpectrum(photon_energies)

    rad = np.array(fr.GetTotalSpectrum())  # this is (dN/dE vs E, units: 1/ erg / cm^2 / s vs TeV)

    sed=rad[:,1]

    # Get back to the world of units
    sed *= u.erg**-1 * u.cm**-2 *u.s **-1

    return sed.to("1 / (cm2 eV s)").reshape(energy_shape)

Now that we have our function, we can create the template model

In [3]:
def make_template_spectral_model(gamera_model,energy_values, index_values,effic_values, filename):

    on_region = CircleSkyRegion(center=SkyCoord.from_name('Crab'), radius=0.1*u.deg)

    energy_axis=MapAxis.from_edges(energy_values,name="energy_true", interp='log')
    efficiency_axis=MapAxis.from_edges(effic_values,name="effic", interp='log')
    spectral_index_axis=MapAxis.from_edges(index_values,name="index")

    region_geom=RegionGeom(on_region,axes=[energy_axis,efficiency_axis,spectral_index_axis])

    template_map=np.empty((spectral_index_axis.nbin,efficiency_axis.nbin,energy_axis.nbin))

    for (i, j) in itertools.product(range(efficiency_axis.nbin),range(spectral_index_axis.nbin)):
        template_map[j,i:]=gamera_model(energy_axis.center,
                                       spectral_index_axis.center[j].to_value(''),
                                       efficiency_axis.center[i].to_value(''))

    template_region_map=RegionNDMap(region_geom,data=template_map,unit=u.eV**-1*u.cm**-2*u.s**-1)
    template_spectral_model=TemplateNDSpectralModel(template_region_map, filename=filename)

    return template_spectral_model


So we can create the model and save it to a file

In [ ]:
index_array = np.linspace(1.5, 4, 20)
effic_array = np.logspace(-3, 1, 40)
energy_array = np.logspace(-1,2,50)*u.TeV

filename = 'models/models_crab_gamera_template.fits.gz'
template_model=make_template_spectral_model(gamera_model,energy_array, index_array, effic_array, filename)


In [ ]:
template_model.write()
target_postion = SkyCoord.from_name("Crab Nebula")
spatial_model = PointSpatialModel.from_position(target_postion)
model = SkyModel(spectral_model=template_model, spatial_model=spatial_model, name='crab')
Models([model]).write('models/models_crab_gamera.yaml', overwrite=True)

Check that it worked

In [ ]:
template_model.parameters['effic'].value = 1
template_model.parameters['index'].value = 2


In [ ]:
import matplotlib.pyplot as plt

ax = template_model.plot(energy_bounds=[0.1,100]*u.TeV,sed_type='e2dnde')
ax.loglog(energy_array, ((energy_array**2)*gamera_model(energy_array, 2,1)).to('erg cm-2 s-1'), ls ='--')

# Approach 2: Subclassing
## We won't use this today but it is here for reference

Below is the implementation of the `GameraSpectralModel` class. It is a subclass of `SpectralModel`. With every evaluation, it evolves a population of electrons in a time-independent environment (i.e. fixed magnetic field and radiation field), takes the output electron spectrum and calculates the gamma-ray spectrum from this.

As the radiation field, we use the model from [Popescu et al. 2017](https://doi.org/10.1093/mnras/stx1282) together with the CMB spectrum, in this case evaluated at the position of the crab nebula. The data of the model can also be found [here](http://cdsarc.u-strasbg.fr/viz-bin/Cat?J/MNRAS/470/2539#/browse). Any other model or radiation spectrum is also possible. It is read here from a txt file with the energies and energy densities of the radiation.

The model has a number of parameters related to the modelled source. Many of these should not be fit, but just be set at the model instantiation and frozen. 

Also, Gamera does not work with astropy units. Therefore, the input quantities to the `GameraSpectralModel` need to be internally stripped of their units for the Gamera calculations. The units are then reattached to the output.

For more details about the Gamera code and methods, see [here](http://libgamera.github.io/GAMERA/docs/time_independent_modeling.html).


In [ ]:
class GameraSpectralModel(SpectralModel):
    """Spectral model from GAMERA synchrotron and IC emission.
    A power law with cutoff is assumed for the electron spectrum.
    To limit the span of the parameters space, we fit the log10 of the parameters
    whose range is expected to cover several orders of magnitudes.

    Some of the parameters (like the distance) should not be fit, but are there just to evaluate the model.
    """

    tag = ["GAMERA", "g"]
    # parameters that we actually might want to fit
    index = Parameter("index", 2.5, min=1.5, max=4.0) #it's positive: E^-index
    log10_effic = Parameter("log10_effic", -2, min=-8, max=0)
    log10_E_min = Parameter("log10_E_min", -5, min=-9, max=0,frozen=True) # in TeV
    log10_E_cut = Parameter("log10_E_cut", 4, min=0, max=6,frozen=True)  # in TeV
    log10_B = Parameter("log10_B", -5, min=-8, max=-2,frozen=True) #in Gauss

    # parameters that we should not fit but just provide
    L_tot = Parameter("L_tot", "1e43 erg s-1", min=1e30, max=1e50, frozen=True) # in ergs/s
    d = Parameter("d", "2 kpc", min=1, max=10, frozen=True) # in kpc
    age = Parameter("age", "3e4 yr", min=1e3, max=1e6, frozen=True)


    @staticmethod
    def evaluate(
        energy,
        log10_B,
        index,
        log10_effic,
        log10_E_min,
        log10_E_cut,
        L_tot,
        d,
        age,
    ):
        # conversions
        B = (10 ** log10_B) * u.G
        effic = 10 ** log10_effic
        E_min = (10 ** log10_E_min) * u.TeV
        E_cut = (10 ** log10_E_cut) * u.TeV

        # Get the relevant GAMERA modules
        fu = gp.Utils()
        fp = gp.Particles()
        fr = gp.Radiation()
        fp.ToggleQuietMode()
        fp.SetSolverMethod(1)
        fr.ToggleQuietMode()

        # GAMERA doesn't know about astropy units
        # so we need to get everything in the right unit
        # and as a normal float and not a quantity
        b_field = B.to_value('G')
        t_max = age.to_value('yr')
        distance = d.to_value('pc')
        e_cut = np.log10(E_cut.to_value('erg'))
        e_min = np.log10(E_min.to_value('erg'))
        norm = (effic * L_tot).to_value('erg s-1')
        energy_shape=energy.shape
        photon_energies = energy.flatten().to_value('erg')
        index = index.value
        # Define the electron energy range and the injected spectrum
        # We want to go beyond the cutoff so we use two orders of magnitude more than e_cut
        energy_electrons_edges = np.logspace(e_min, e_cut+2, 100+1) # it's in ergs

        padded = False

        
        # GAMERA removes the highest and lowest energy bin so we need to pad it so that our choice of E_min is the true one
        step_log = np.diff(np.log10(energy_electrons_edges))[0]
        energy_electrons_edges = np.append(energy_electrons_edges[0]*10**-step_log, energy_electrons_edges)
        energy_electrons_edges = np.append(energy_electrons_edges, energy_electrons_edges[-1]*10**step_log)
        padded = True

        if padded:
            range = slice(1,-1, 1)
        else:
            range = slice(None, None, 1)

        energy_electrons = MapAxis.from_edges(energy_electrons_edges, interp='log', unit='erg')
        exp_cutoff = np.exp(-(energy_electrons.center.to('erg')/E_cut.to('erg')).value)
        power_law = (energy_electrons.center.to_value('erg') ** -index)*exp_cutoff

        # Normalize to total power
        # Note that if we have padded, the range is different
        power_law *= norm / fu.Integrate(list(zip(energy_electrons.center.to_value('erg')[range], power_law[range] * energy_electrons.center.to_value('erg')[range])))

        # Bundle together in the way GAMERA wants
        power_law_spectrum = np.array(list(zip(energy_electrons.center.to_value('erg'), power_law)))

        #Read the radiation field spectrum, in this case the Popescu et al. 2017 model + CM at the crab nebula position. 
        rad_field = np.loadtxt("models/radiation_field.txt")

        RADIATION_FIELD = list(zip(rad_field[:,0], rad_field[:,1]))

        # Set everything that is left
        fp.AddArbitraryTargetPhotons(RADIATION_FIELD)
        fp.SetCustomInjectionSpectrum(power_law_spectrum)  # the injection rate is constant and given by the normalization of injected PL
        fr.AddArbitraryTargetPhotons(RADIATION_FIELD)
        fr.SetDistance(distance)
        fp.SetBField(b_field)
        fr.SetBField(b_field)


        #Run the evolution of the electron spectrum and set the output as parent population for radiation calculation
        fp.SetAge(t_max)
        fp.CalculateElectronSpectrum()
        sp = np.array(fp.GetParticleSpectrum())
        fr.SetElectrons(sp)

        # compute the gamma-ray spectrum on the points given by the data
        fr.CalculateDifferentialPhotonSpectrum(photon_energies)
        rad = np.array(fr.GetTotalSpectrum())  # this is (dN/dE vs E, units: 1/ erg / cm^2 / s vs TeV)

        sed=rad[:,1]

        # Get back to the world of units
        sed *= u.erg**-1 * u.cm**-2 *u.s **-1

        return sed.to("1 / (cm2 eV s)").reshape(energy_shape)

In [ ]:
SPECTRAL_MODEL_REGISTRY.append(GameraSpectralModel)

In [ ]:
gamera_custom_spectral_model=GameraSpectralModel()